In [ ]:
import warnings

import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from pandas.core.interchange.dataframe_protocol import DataFrame
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import PowerTransformer, TargetEncoder, Normalizer

In [ ]:
train: DataFrame = pd.read_csv('final_proj_data.csv')
valid = pd.read_csv('final_proj_test.csv')

valid: DataFrame = valid[valid.columns[valid.isna().mean().lt(0.35)]]
train: DataFrame = train[train.columns[train.isna().mean().lt(0.35)]]
train = train.dropna(thresh=train.shape[1]- (int(len(train.columns) * 0.65)), axis=0)

target = train.pop('y')

In [ ]:
model = make_pipeline(

    # Заповнюємо відсутні категоріальні ознаки
    make_column_transformer(
        (
            SimpleImputer(strategy='most_frequent').set_output(transform='pandas'),
            make_column_selector(dtype_include=object)
        ),
        remainder='passthrough',
        n_jobs=-1
    ).set_output(transform='pandas'),

    # Заповнюємо відсутні числові ознаки
    make_column_transformer(
        (
            SimpleImputer().set_output(transform='pandas'),
            make_column_selector(dtype_include=np.number)
        ),
        remainder='passthrough',
        n_jobs=-1
    ).set_output(transform='pandas'),

    # Кодуємо категоріальні ознаки
    make_column_transformer(
        (
            TargetEncoder(random_state=42, cv=10).set_output(transform='pandas'),
            make_column_selector(dtype_include=object)
        ),
        remainder='passthrough',
        n_jobs=-1
    ).set_output(transform='pandas'),

    # Кодуємо числові ознаки
    make_column_transformer(
        (
            TargetEncoder(random_state=42, cv=10).set_output(transform='pandas'),
            make_column_selector(dtype_include=np.number)
        ),
        remainder='passthrough',
        n_jobs=-1
    ).set_output(transform='pandas'),

    # Відбираємо найбільш важливі ознаки
    SelectKBest().set_output(transform='pandas'),

    # Нормалізуємо розподілення даних
    PowerTransformer().set_output(transform='pandas'),

    # Балансуємо класи даних
    SMOTE(random_state=42),

    # Нормалізуємо дані
    Normalizer(),

    GradientBoostingClassifier(random_state=42),
)


In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    cv_results = cross_val_score(
        estimator=model,
        X=train,
        y=target,
        scoring='balanced_accuracy',
        cv=10,
        n_jobs=-1)


In [ ]:
print(cv_results.mean())

[0.86498674 0.83660477 0.8602122  0.82944297 0.84407604 0.83400241
 0.81810276 0.83415613 0.84184682 0.85580952]
0.8419240361321556


0.8419240361321556

In [ ]:
model.fit(train, target)

output = pd.DataFrame({'index': valid.index, 'y': model.predict(valid)})
output.to_csv('submission.csv', index=False)